# Basic outline of polymer loading with OpenFF and our tools

## Basic imports

In [1]:
# Supressing annoying warnings (!must be done first!)
import warnings
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)
LOGGER = logging.getLogger(__name__)

## Defining paths to PDB and monomer template files

In [2]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import assemble_path

mol_name = 'polyvinylchloride' # this should be swapped out for the target molecule (provided files exist)
mol_class = 'simple_polymers'

src_dir  = Path('cleaned_structures')
pdb_dir  = src_dir / 'pdbs'  / mol_class
mono_dir = src_dir / 'monos' / mol_class

pdb_path  = assemble_path(pdb_dir , mol_name, extension='pdb')
mono_path = assemble_path(mono_dir, mol_name, extension='json')

for path in (pdb_path, mono_path):
    assert(path.exists()) # make sure 

## Create Monomer Group

In [ ]:
from polymerist.polymers.monomers import MonomerGroup

grp = MonomerGroup.from_file(mono_path)
for resname, mol in grp.iter_rdmols():
    print(resname)
    display(mol)

## Load topology with substructures, generate partition

In [ ]:
from openff.toolkit import Topology, Molecule
from polymerist.mdtools.openfftools import topology
from polymerist.mdtools.openfftools.partition import partition


offtop = Topology.from_pdb(pdb_path, _custom_substructures=grp.monomers)
was_partitioned = partition(offtop)
assert(was_partitioned)

offmol = topology.get_largest_offmol(offtop)
display(offmol)

### Assign partial charges and periodic box

In [ ]:
import numpy as np


offmol.assign_partial_charges(partial_charge_method='gasteiger') # assign quick-and-easy dummy charges as prrof-of-concept

conf = offmol.conformers[0]
conf_vals, conf_units = conf.magnitude, conf.units
box_vectors = np.diag(np.ptp(conf_vals, axis=0)) * conf_units

### Relabel partitioned molecule residues (necessary for GROMACS output)

In [5]:
unique_res_nums = sorted(set(atom.metadata['residue_number'] for atom in offmol.atoms))
res_num_remap = {
    res_num : i + 1 # new number must be non-zero
        for (i, res_num) in enumerate(unique_res_nums)
}

for atom in offmol.atoms:
    old_res_num = atom.metadata.pop('residue_number')
    atom.metadata['residue_number'] = res_num_remap[old_res_num]

### TODO : include RCT partial charging protocol (toolkit assignment is fine for now for small molecules)

## Load force field and create Interchange for format interconversion

In [ ]:
from openff.toolkit import ForceField
from openff.interchange import Interchange


groname = 'gromacs_demo'
grodir = Path('GROMACS_demo')
grodir.mkdir(exist_ok='true')

ff = ForceField('openff-2.0.0.offxml')
inc = Interchange.from_smirnoff(force_field=ff, topology=offmol.to_topology(), charge_from_molecules=[offmol])
inc.box = box_vectors
inc.to_gromacs(prefix=str(grodir / groname))